### SocialAL Models
# Data simulation using best-fit parameters - multiple subjects
KLS 8.30.19; update 7.7.22; updated 10.31.22
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
from scipy.stats import norm
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run ../common_functions.ipynb

In [4]:
run ../single_alpha_decay_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    
    alpha = params[0]
    beta = params[1]
    decay = params[2]

    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], alpha)
        # then update value
        EVs[t] = update_value(Probs[t]) 
        
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(t)
        for x in options:
            Probs[x] = Probs[x] + decay * (0.5 - Probs[x])
            EVs[x] =  update_value(Probs[x]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
# check simulation 
#sim_data(15,[.2, 2, .2])

### Pull in best-fit parameters from participants

In [11]:
dt1 = pd.read_csv('../../../output/single_alpha_with_decay_model_params.csv')
dt2 = pd.read_csv('../../../output/missing_single_alpha_with_decay_model_params.csv')
dt = pd.concat([dt1, dt2], ignore_index = True)
print(dt)

          id     alpha          beta     decay      X.LLH       -LLH
0   sub-2013  0.004415  2.000000e+01  0.032207  46.686238        NaN
1   sub-2007  0.212075  3.359906e-01  0.000000  51.644795        NaN
2   sub-2006  0.000000  1.000000e-10  1.000000  60.996952        NaN
3   sub-2012  1.000000  1.499341e-01  0.621076  61.447734        NaN
4   sub-2004  0.173984  1.224196e+00  0.010163  35.031480        NaN
..       ...       ...           ...       ...        ...        ...
58  sub-2021  0.935232  1.868886e-01  0.000000        NaN  54.493332
59  sub-2037  1.000000  2.674687e-01  0.070692        NaN  49.838614
60  sub-2023  0.482881  6.157203e-01  0.000000        NaN  34.813597
61  sub-2022  0.536960  8.943738e-03  1.000000        NaN  60.995161
62  sub-2036  0.001895  2.000000e+01  1.000000        NaN  62.186124

[63 rows x 6 columns]


In [12]:
alphas = dt['alpha']
betas = dt['beta']
decays = dt['decay']
parent_sub = dt['id']
print(decays)

0     0.032207
1     0.000000
2     1.000000
3     0.621076
4     0.010163
        ...   
58    0.000000
59    0.070692
60    0.000000
61    1.000000
62    1.000000
Name: decay, Length: 63, dtype: float64


### Simulate data

In [13]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha', 'Beta', 'Decay'])
for p in range(len(alphas)):
    for q in range(1,11):
        dt = sim_data(15, [alphas[p], betas[p], decays[p]])
        dt['Subject'] = parent_sub[p] + '_' + str(q)
        dt['Alpha'] = [alphas[p]] * 45
        dt['Beta'] = [betas[p]] * 45
        dt['Decay'] = [decays[p]] * 45
        dt = pd.DataFrame(dt)
        data = pd.concat([data, dt])

data.to_csv(path_or_buf = '../../../output/simulation/part_params/sim_1alpha_decay_model_data.csv', index = False)